In [1]:
import sys,os
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

In [2]:
def get_lig_coord(filename):
    '''
    Get all heavy atom coordinates and return a coordinate matrix

    '''
    f1 = open(filename, 'r')

    List = []
    for line in f1.readlines():
        if line.startswith(('HETATM', 'ATOM')) and line[77]!='H':
            coord_x, coord_y, coord_z = line[27:38], line[38:46], line[46:54]
            result = [float(coord_x), float(coord_y), float(coord_z)]
            List.append(result)
    f1.close()
    List = np.asarray(List)
    return List

In [3]:
def get_beta_info(filename):
    '''
    Get beta atom coordinates and the corresponding beta atom scores

    '''
    f1 = open(filename, 'r')

    List = []
    Score = []
    for line in f1.readlines():
        if line.startswith(('HETATM', 'ATOM')) and line[77]!='H':
            coord_x, coord_y, coord_z = line[27:38], line[38:46], line[46:54]
            result = [float(coord_x), float(coord_y), float(coord_z)]
            List.append(result)
            score = float(line[61:66])
            Score.append(score)
    f1.close()
    List = np.asarray(List)
    Score = np.asarray(Score)
    return List, Score


In [4]:
def calc_betaScore_and_ligCover(lig_file, beta_file):
    lig_coords = get_lig_coord(lig_file)
    beta_coords, beta_scores = get_beta_info(beta_file)
    result1 = cdist(beta_coords, lig_coords)
    score = np.sum(beta_scores[np.min(result1, axis=1)<=1.6])
    result2 = cdist(lig_coords, beta_coords)
    lig_cover_coords = lig_coords[np.min(result2, axis=1)<=1.6]
    return round(score,3),round(len(lig_cover_coords)/len(lig_coords),3)

In [5]:
df = pd.read_csv('feature_set/poses_data_Lin_F9_Score.csv')

In [6]:
df

,ligand_id,compound_stem,compound_name,new_smile,o_index,Lin_F9_Score
0,1,A00366875,NaN,Cc1cccc(NC(=O)N2CCC3(CC2)CC(CC(=O)N(C)C)c2cccc...,1,-9.203915
1,1,A00366875,NaN,Cc1cccc(NC(=O)N2CCC3(CC2)CC(CC(=O)N(C)C)c2cccc...,2,-8.873600
2,1,A00366875,NaN,Cc1cccc(NC(=O)N2CCC3(CC2)CC(CC(=O)N(C)C)c2cccc...,3,-8.815475
3,1,A00366875,NaN,Cc1cccc(NC(=O)N2CCC3(CC2)CC(CC(=O)N(C)C)c2cccc...,4,-8.577186
4,1,A00366875,NaN,Cc1cccc(NC(=O)N2CCC3(CC2)CC(CC(=O)N(C)C)c2cccc...,5,-8.576674
...,...,...,...,...,...,...
14305,2866,M65097687,PAS,Nc1ccc(C(=O)O)c(O)c1,1,-3.959940
14306,2866,M65097687,PAS,Nc1ccc(C(=O)O)c(O)c1,2,-3.936136
14307,2866,M65097687,PAS,Nc1ccc(C(=O)O)c(O)c1,3,-3.892714
14308,2866,M65097687,PAS,Nc1ccc(C(=O)O)c(O)c1,4,-3.872353


In [16]:
from tqdm import tqdm

lig_Cover,betaScore,Is,Js=[],[],[],[]
beta_file='beta_Atoms/4rtf_site1.pdb'

for i,j in list((zip(df.ligand_id.tolist(),df.o_index.tolist()))):
    lig_file='pose_site1/%d_%d.pdb'%(i,j) 
    a1,a2 = calc_betaScore_and_ligCover(lig_file,beta_file)
    print(i,j,a1,a2)
    Is.append(i)
    Js.append(j)
    betaScore.append(a1)
    lig_Cover.append(a2)
return a1,a2

1 1 -5.8 0.613
1 2 -5.9 0.613
1 3 -5.8 0.581
1 4 -6.1 0.613
1 5 -6.3 0.613
2 1 -6.4 0.826
2 2 -6.0 0.826
2 3 -8.4 0.826
2 4 -8.5 0.826
2 5 -3.7 0.696
3 1 -5.2 0.857
3 2 -6.1 0.81
3 3 -2.9 0.619
3 4 -4.3 0.667
3 5 -2.3 0.429
4 1 -7.0 0.593
4 2 -7.0 0.778
4 3 -6.2 0.704
4 4 -6.2 0.741
4 5 -6.5 0.815
5 1 -6.4 0.833
5 2 -7.3 0.875
5 3 -6.9 0.833
5 4 -7.0 0.75
5 5 -6.0 0.708
6 1 -7.0 0.667
6 2 -9.1 0.6
6 3 -6.6 0.733
6 4 -6.2 0.733
6 5 -5.9 0.633
7 1 -7.2 0.84
7 2 -8.5 0.68
7 3 -7.1 0.8
7 4 -3.2 0.4
7 5 -1.9 0.24
8 1 -8.7 0.783
8 2 -4.9 0.696
8 3 -5.3 0.826
8 4 -5.1 0.739
8 5 -4.4 0.783
9 1 -5.4 0.6
9 2 -3.7 0.48
9 3 -4.1 0.48
9 4 -5.4 0.56
9 5 -4.3 0.6
10 1 -8.9 0.491
10 2 -8.7 0.509
10 3 -3.8 0.211
10 4 -5.5 0.386
10 5 -6.3 0.316
11 1 -7.8 0.875
11 2 -8.2 0.875
11 3 -5.8 0.667
11 4 -7.1 0.792
11 5 -6.9 0.75
12 1 -7.2 0.87
12 2 -7.2 0.783
12 3 -7.2 0.739
12 4 -7.5 0.783
12 5 -5.0 0.783
13 1 -6.4 0.469
13 2 -6.6 0.531
13 3 -7.0 0.531
13 4 -6.6 0.531
13 5 -5.3 0.5
14 1 -10.1 0.92
14 2 -7.5 0

129 2 -7.0 0.87
129 3 -7.4 0.957
129 4 -6.4 0.826
129 5 -6.4 0.826
130 1 -6.2 0.833
130 2 -5.9 0.889
130 3 -5.2 0.833
130 4 -6.2 0.889
130 5 -6.1 0.889
131 1 -5.0 0.833
131 2 -8.2 0.875
131 3 -3.7 0.708
131 4 -3.7 0.667
131 5 -4.1 0.75
132 1 -6.7 0.913
132 2 -6.3 0.739
132 3 -6.3 0.826
132 4 -6.7 0.783
132 5 -6.7 0.739
133 1 -7.1 0.864
133 2 -5.3 0.818
133 3 -7.5 0.818
133 4 -4.9 0.773
133 5 -8.1 0.909
134 1 -6.6 0.833
134 2 -6.6 0.875
134 3 -6.6 0.792
134 4 -5.8 0.792
134 5 -5.8 0.792
135 1 -8.8 1.0
135 2 -8.8 1.0
135 3 -7.7 0.895
135 4 -5.5 0.737
135 5 -7.0 0.842
136 1 -6.2 0.808
136 2 -6.2 0.808
136 3 -5.3 0.692
136 4 -4.5 0.615
136 5 -3.8 0.462
137 1 -6.3 0.531
137 2 -6.3 0.562
137 3 -7.5 0.531
137 4 -6.4 0.625
137 5 -6.4 0.625
138 1 -8.4 0.96
138 2 -7.1 0.72
138 3 -4.1 0.48
138 4 -6.5 0.6
138 5 -6.5 0.68
139 1 -5.0 0.762
139 2 -6.8 0.81
139 3 -2.7 0.619
139 4 -4.5 0.619
139 5 -2.3 0.476
140 1 -2.9 0.25
140 2 -3.5 0.469
140 3 -1.8 0.344
140 4 -3.3 0.25
140 5 -5.2 0.562
141 1 -3.7 0

253 4 -1.8 0.6
253 5 -6.1 1.0
254 1 -6.4 0.85
254 2 -8.1 0.9
254 3 -5.4 0.85
254 4 -7.6 0.85
254 5 -4.3 0.85
255 1 -8.2 0.774
255 2 -9.1 0.903
255 3 -8.6 0.839
255 4 -6.7 0.71
255 5 -5.4 0.677
256 1 -3.2 0.727
256 2 -4.4 0.636
256 3 -8.5 0.864
256 4 -3.4 0.545
256 5 -5.4 0.636
257 1 -2.8 0.667
257 2 -3.9 0.667
257 3 -2.8 0.476
257 4 -4.2 0.81
257 5 -3.4 0.571
258 1 -6.6 0.72
258 2 -1.8 0.32
258 3 -3.6 0.64
258 4 -7.5 0.72
258 5 -1.8 0.4
259 1 -10.3 0.846
259 2 -11.7 0.885
259 3 -8.7 0.808
259 4 -9.5 0.885
259 5 -10.7 0.846
260 1 -7.0 0.808
260 2 -6.2 0.808
260 3 -4.4 0.731
260 4 -5.8 0.769
260 5 -4.8 0.654
261 1 -7.9 0.633
261 2 -7.2 0.767
261 3 -7.3 0.533
261 4 -6.3 0.733
261 5 -9.9 0.767
262 1 -4.3 0.696
262 2 -8.9 0.739
262 3 -3.7 0.652
262 4 -5.2 0.696
262 5 -5.7 0.522
263 1 -10.6 0.885
263 2 -3.1 0.5
263 3 -6.5 0.731
263 4 -3.2 0.615
263 5 -5.6 0.731
264 1 -6.2 0.667
264 2 -2.8 0.375
264 3 -3.6 0.542
264 4 -6.2 0.875
264 5 -5.4 0.833
265 1 -6.3 0.591
265 2 -4.1 0.636
265 3 -4.0 0.

369 1 -7.1 0.8
369 2 -7.2 0.667
369 3 -7.1 0.8
369 4 -7.8 0.867
369 5 -7.1 0.633
370 1 -6.7 0.68
370 2 -5.9 0.76
370 3 -6.2 0.64
370 4 -9.0 0.92
370 5 -7.8 0.84
371 1 -9.1 0.913
371 2 -6.6 0.783
371 3 -5.7 0.739
371 4 -5.5 0.783
371 5 -6.1 0.87
372 1 -9.0 0.857
372 2 -9.0 0.857
372 3 -3.8 0.607
372 4 -4.8 0.536
372 5 -6.9 0.607
373 1 -5.8 0.75
373 2 -6.6 0.75
373 3 -5.0 0.625
373 4 -9.4 0.875
373 5 -6.0 0.688
374 1 -7.0 0.88
374 2 -7.0 0.88
374 3 -7.1 0.88
374 4 -7.3 0.88
374 5 -6.6 0.76
375 1 -7.6 0.933
375 2 -6.9 0.667
375 3 -7.6 0.9
375 4 -9.5 0.667
375 5 -4.7 0.5
376 1 -7.4 0.704
376 2 -3.3 0.444
376 3 -5.6 0.815
376 4 -6.0 0.704
376 5 -7.4 0.741
377 1 -6.3 0.69
377 2 -6.7 0.759
377 3 -6.2 0.655
377 4 -8.0 0.759
377 5 -8.1 0.897
378 1 -7.6 0.767
378 2 -8.7 0.8
378 3 -7.6 0.667
378 4 -7.6 0.733
378 5 -8.0 0.633
379 1 -5.7 0.833
379 2 -9.3 0.917
379 3 -1.8 0.375
379 4 -4.6 0.625
379 5 -10.3 0.833
380 1 -4.3 0.524
380 2 -4.3 0.524
380 3 -5.4 0.714
380 4 -3.7 0.619
380 5 -6.7 0.857
381

491 5 -8.4 0.581
492 1 -7.6 0.821
492 2 -8.1 0.821
492 3 -8.5 0.75
492 4 -8.2 0.714
492 5 -7.8 0.75
493 1 -3.1 0.385
493 2 -5.8 0.538
493 3 -1.9 0.282
493 4 -7.1 0.718
493 5 -6.3 0.436
494 1 -4.1 0.433
494 2 -5.0 0.5
494 3 -4.7 0.567
494 4 -6.7 0.533
494 5 -2.8 0.333
495 1 -5.9 0.667
495 2 -4.0 0.5
495 3 -5.9 0.792
495 4 -8.1 0.792
495 5 -9.0 0.75
496 1 -5.4 0.515
496 2 -5.4 0.545
496 3 -5.4 0.545
496 4 -5.9 0.636
496 5 -5.7 0.485
497 1 -14.6 0.623
497 2 -14.4 0.656
497 3 -14.9 0.639
497 4 -14.0 0.607
497 5 -13.5 0.672
498 1 -7.0 0.773
498 2 -5.4 0.682
498 3 -7.4 0.773
498 4 -7.9 0.727
498 5 -6.5 0.818
499 1 -1.8 0.391
499 2 -3.2 0.391
499 3 -3.2 0.565
499 4 -1.8 0.391
499 5 -5.8 0.739
500 1 -7.0 0.724
500 2 -8.4 0.862
500 3 -8.4 0.862
500 4 -7.0 0.655
500 5 -8.0 0.793
501 1 -8.6 0.806
501 2 -8.6 0.806
501 3 -8.8 0.71
501 4 -9.5 0.839
501 5 -9.5 0.871
502 1 -8.9 0.737
502 2 -9.3 0.737
502 3 -9.9 0.684
502 4 -10.6 0.658
502 5 -7.5 0.605
503 1 -6.9 0.88
503 2 -6.3 0.88
503 3 -5.9 0.96
50

...

2481 3 -9.1 0.917
2481 4 -9.5 0.875
2481 5 -6.4 0.792
2482 1 -9.8 0.857
2482 2 -4.6 0.571
2482 3 -9.3 0.786
2482 4 -9.1 0.857
2482 5 -2.0 0.464
2483 1 -4.3 0.818
2483 2 -8.0 0.818
2483 3 -5.6 0.773
2483 4 -8.4 0.773
2483 5 -3.0 0.591
2484 1 -8.1 0.784
2484 2 -7.9 0.676
2484 3 -9.0 0.568
2484 4 -6.0 0.541
2484 5 -4.6 0.432
2485 1 -3.1 0.5
2485 2 -3.1 0.5
2485 3 -3.1 0.5
2485 4 -8.4 0.875
2485 5 -10.3 0.917
2486 1 -4.5 0.571
2486 2 -7.0 0.607
2486 3 -6.7 0.643
2486 4 -5.4 0.5
2486 5 -5.8 0.75
2487 1 -9.0 1.0
2487 2 -5.6 0.786
2487 3 -5.0 0.929
2487 4 -6.9 0.929
2487 5 -7.8 0.857
2488 1 -7.7 0.697
2488 2 -8.7 0.636
2488 3 -2.4 0.364
2488 4 -2.3 0.273
2488 5 -9.7 0.758
2489 1 -8.2 1.0
2489 2 -9.1 0.955
2489 3 -9.6 0.955
2489 4 -10.0 0.909
2489 5 -7.1 0.818
2490 1 -8.4 0.704
2490 2 -8.7 0.778
2490 3 -9.8 0.815
2490 4 -7.8 0.63
2490 5 -2.7 0.407
2491 1 -4.7 0.5
2491 2 -4.9 0.5
2491 3 -7.9 0.472
2491 4 -7.9 0.472
2491 5 -7.9 0.472
2492 1 -6.3 0.514
2492 2 -4.6 0.514
2492 3 -10.1 0.829
2492 4 

2599 3 -7.0 0.567
2599 4 -7.0 0.567
2599 5 -7.4 0.667
2600 1 -7.6 0.769
2600 2 -5.8 0.808
2600 3 -9.1 0.885
2600 4 -7.6 0.846
2600 5 -6.2 0.846
2601 1 -7.5 0.786
2601 2 -8.0 0.821
2601 3 -7.1 0.929
2601 4 -8.5 0.786
2601 5 -6.5 0.857
2602 1 -8.0 0.629
2602 2 -3.7 0.371
2602 3 -3.8 0.343
2602 4 -3.1 0.343
2602 5 -3.7 0.343
2603 1 -5.7 0.609
2603 2 -6.6 0.652
2603 3 -2.1 0.304
2603 4 -5.6 0.696
2603 5 -5.5 0.739
2604 1 -7.4 0.793
2604 2 -7.2 0.724
2604 3 -8.2 0.828
2604 4 -9.4 0.793
2604 5 -5.9 0.655
2605 1 -6.6 0.731
2605 2 -6.6 0.692
2605 3 -6.6 0.731
2605 4 -6.6 0.731
2605 5 -6.6 0.692
2606 1 -4.1 0.812
2606 2 -2.8 0.75
2606 3 -5.1 0.938
2606 4 -4.6 1.0
2606 5 -6.1 0.938
2607 1 -5.8 0.568
2607 2 -3.5 0.351
2607 3 -4.0 0.351
2607 4 -3.5 0.351
2607 5 -4.5 0.405
2608 1 -5.9 0.559
2608 2 -5.9 0.559
2608 3 -5.1 0.559
2608 4 -4.7 0.5
2608 5 -5.5 0.676
2609 1 -7.5 0.742
2609 2 -6.7 0.71
2609 3 -7.5 0.774
2609 4 -6.7 0.774
2609 5 -6.7 0.742
2610 1 -8.0 0.759
2610 2 -8.4 0.862
2610 3 -9.0 0.82

2714 4 -6.8 0.69
2714 5 -11.1 0.793
2715 1 -6.0 0.947
2715 2 -7.0 0.895
2715 3 -5.6 0.947
2715 4 -4.8 0.737
2715 5 -5.3 0.737
2716 1 -6.4 0.462
2716 2 -8.7 0.718
2716 3 -7.0 0.564
2716 4 -7.0 0.564
2716 5 -5.5 0.436
2717 1 -7.1 0.792
2717 2 -8.4 0.958
2717 3 -6.1 0.667
2717 4 -10.5 0.917
2717 5 -6.2 0.792
2718 1 -5.1 0.56
2718 2 -7.6 0.88
2718 3 -5.4 0.72
2718 4 -7.5 0.84
2718 5 -9.6 0.92
2719 1 -6.9 0.826
2719 2 -7.2 0.783
2719 3 -6.1 0.783
2719 4 -5.2 0.826
2719 5 -7.5 0.913
2720 1 -4.7 0.382
2720 2 -4.7 0.412
2720 3 -4.8 0.676
2720 4 -4.8 0.676
2720 5 -9.7 0.706
2721 1 -8.0 0.667
2721 2 -7.0 0.694
2721 3 -9.5 0.75
2721 4 -7.8 0.639
2721 5 -8.0 0.667
2722 1 -7.8 0.893
2722 2 -7.7 0.714
2722 3 -7.5 0.714
2722 4 -8.2 0.714
2722 5 -5.4 0.5
2723 1 -19.0 0.47
2723 2 -19.1 0.47
2723 3 -19.9 0.478
2723 4 -17.7 0.452
2723 5 -18.2 0.452
2724 1 -8.3 0.486
2724 2 -8.3 0.486
2724 3 -8.2 0.459
2724 4 -8.8 0.486
2724 5 -9.8 0.676
2725 1 -9.0 0.818
2725 2 -3.5 0.364
2725 3 -3.5 0.364
2725 4 -3.5 0.

2836 4 -9.9 0.667
2836 5 -7.5 0.606
2837 1 -10.1 0.667
2837 2 -7.9 0.636
2837 3 -8.5 0.697
2837 4 -10.1 0.697
2837 5 -6.3 0.576
2838 1 -9.1 1.0
2838 2 -9.1 1.0
2838 3 -7.0 1.0
2838 4 -6.4 0.895
2838 5 -5.8 0.947
2839 1 -5.7 0.76
2839 2 -6.6 0.8
2839 3 -3.7 0.48
2839 4 -6.5 0.76
2839 5 -7.3 0.8
2840 1 -2.6 0.6
2840 2 -5.6 0.65
2840 3 -8.5 0.75
2840 4 -7.8 0.75
2840 5 -5.1 0.75
2841 1 -6.1 0.875
2841 2 -4.3 0.75
2841 3 -3.8 0.75
2841 4 -5.7 0.875
2841 5 -4.3 0.75
2842 1 -4.0 0.375
2842 2 -4.0 0.375
2842 3 -4.6 0.406
2842 4 -4.0 0.375
2842 5 -5.0 0.438
2843 1 -6.7 0.645
2843 2 -9.9 0.677
2843 3 -5.9 0.581
2843 4 -5.8 0.516
2843 5 -4.4 0.452
2844 1 -5.6 0.5
2844 2 -4.9 0.542
2844 3 -7.4 0.75
2844 4 -6.6 0.667
2844 5 -6.0 0.583
2845 1 -5.7 0.929
2845 2 -1.8 0.643
2845 3 -1.6 0.571
2845 4 -5.0 0.929
2845 5 -3.8 0.857
2846 1 -4.3 0.455
2846 2 -10.6 0.727
2846 3 -4.6 0.576
2846 4 -3.7 0.455
2846 5 -2.7 0.364
2847 1 -5.7 0.5
2847 2 -6.4 0.643
2847 3 -7.8 0.607
2847 4 -6.9 0.536
2847 5 -6.5 0.53

SyntaxError: 'return' outside function (<ipython-input-16-51332c797cac>, line 14)

In [17]:
df_n=pd.DataFrame()
df_n['ligand_id']=Is
df_n['o_index']=Js
df_n['betaScore']=betaScore
df_n['lig_Cover']=lig_Cover

In [19]:
new_df_n = df_n.sort_values(["ligand_id", "o_index"], ascending = (True, True))

In [20]:
new_df_n

,ligand_id,o_index,betaScore,lig_Cover
0,1,1,-5.8,0.613
1,1,2,-5.9,0.613
2,1,3,-5.8,0.581
3,1,4,-6.1,0.613
4,1,5,-6.3,0.613
...,...,...,...,...
14305,2866,1,-4.8,1.000
14306,2866,2,-5.3,0.727
14307,2866,3,-4.8,0.909
14308,2866,4,-4.3,0.909


In [22]:
new_df_n.to_csv('feature_set/betaScore_ligCover.csv')